# Generate data to ML models
Generate the data to ML models. For this example only ficticious data are generated acording a certain formula (also ficticous that try to represent the problem of process that generate an output and this output pass across a tank and then go to another tank)

## Root folder and read env variables

In [1]:
import os
# fix root path to save outputs
actual_path = os.path.abspath(os.getcwd())
list_root_path = actual_path.split('\\')[:-1]
root_path = '\\'.join(list_root_path)
os.chdir(root_path)
print('root path: ', root_path)

root path:  D:\github-mi-repo\Gurobi-ML-tips-modeling


In [2]:
import os
from dotenv import load_dotenv, find_dotenv # package used in jupyter notebook to read the variables in file .env

""" get env variable from .env """
load_dotenv(find_dotenv())

""" Read env variables and save it as python variable """
PROJECT_GCP = os.environ.get("PROJECT_GCP", "")

## RUN

In [3]:
import pandas as pd
import numpy as np
from google.cloud import bigquery
import gcsfs
import pickle

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [4]:
# ### desarrollo

# PROJECT_ID = PROJECT_GCP
# ! gcloud config set project $PROJECT_ID

### 0. Paramaters

In [5]:
len_data = 1000

### 1. Read Master Table

In [6]:
path_master_table = f'config/MasterTable.xlsx'
master_table = pd.read_excel(path_master_table)
master_table

,TAG,FEATURES_NAMES,DESCRIPCION,CLASIFICACION_NAME,CLASIFICACION,USE_ACTUAL_MODEL
0,X1,X1,Variable de entrada al proceso A. Variable Pri...,Primary,P,MLA
1,O1,O1,Variable de entrada al proceso A. No es una va...,Observed,O,MLA
2,O2,O2,Variable de entrada al proceso A. No es una va...,Observed,O,MLA
3,O3,O3,Variable de entrada al proceso A. No es una va...,Observed,O,MLA
4,Y1,Y1,Variable target del proceso A y Variable de en...,Target,T,MLA
5,Z1,Z1,Variable de salida del tanque X y Variable de ...,Secundary,S,MLB
6,X2,X2,Variable de entrada al proceso B. Aparece por ...,Primary,P,MLB
7,O4,O4,Variable de entrada al proceso B. No es una va...,Observed,O,MLB
8,O5,O5,Variable de entrada al proceso B. No es una va...,Observed,O,MLB
9,Y2,Y2,Variable target del proceso B y proceso C (y v...,Target,T,MLB/MLC


### 2. Define data random features and target
Define data random and create a formula to map the features into a target and then generate random data to append to the target

#### 2.1 Process A

In [7]:
list_features_process_a = [
    "X1",
    "O1",
    "O2",
    "03"
]

list_target_process_a = ["Y1"]

In [8]:
##### FEATURES

# generate data features process_a
np.random.seed(42)
values_features_process_a = 10 * np.random.rand(len_data, len(list_features_process_a))

In [9]:
#### TARGET

# define formula to map features into a target. np.exp(-1/data) + np.log(data)
formula_target_process_a = (np.exp(-1/values_features_process_a) + np.log(values_features_process_a)).sum(axis = 1)

# generate noise to add to the formula of the target generated
np.random.seed(52)
noise_target_process_a = 10 * np.random.rand(len_data, len(list_target_process_a))

# generate data target process_a
values_target_process_a  = formula_target_process_a.reshape(formula_target_process_a.shape[0], 1) + noise_target_process_a

In [10]:
# generate dataframe process_a
data_process_a = pd.DataFrame(values_features_process_a, columns = list_features_process_a)
data_process_a[list_target_process_a] = values_target_process_a
data_process_a

,X1,O1,O2,03,Y1
0,3.745401,9.507143,7.319939,5.986585,18.968108
1,1.560186,1.559945,0.580836,8.661761,4.889535
2,6.011150,7.080726,0.205845,9.699099,9.174935
3,8.324426,2.123391,1.818250,1.834045,12.928706
4,3.042422,5.247564,4.319450,2.912291,9.334401
...,...,...,...,...,...
995,8.323635,8.056488,2.243186,2.260791,10.888191
996,8.173053,9.303984,0.953609,4.500586,12.010378
997,3.374543,8.706885,0.835249,2.111259,11.360801
998,7.524683,0.512938,4.925354,4.421070,15.351852


#### 2.2 Process B

In [11]:
list_features_process_b = [
    "Z1",
    "X2",
    "O4",
    "O5"
]

list_target_process_b = ["Y2"]

In [12]:
##### FEATURES

# generate data features process_b
np.random.seed(62)
values_features_process_b = 10 * np.random.rand(len_data, len(list_features_process_b))

In [13]:
#### TARGET

# define formula to map features into a target. np.exp(-3/data) + np.log(data)
formula_target_process_b = (np.exp(-3/values_features_process_b) + np.log(values_features_process_b)).sum(axis = 1)

# generate noise to add to the formula of the target generated
np.random.seed(12)
#noise_target_process_b = 10 * np.random.rand(len_data, len(list_target_process_b))
noise_target_process_b = 10

# generate data target process_b
values_target_process_b  = formula_target_process_b.reshape(formula_target_process_b.shape[0], 1) + noise_target_process_b

In [14]:
# generate dataframe process_b
data_process_b = pd.DataFrame(values_features_process_b, columns = list_features_process_b)
data_process_b[list_target_process_b] = values_target_process_b
data_process_b

,Z1,X2,O4,O5,Y2
0,0.337547,4.891075,8.460851,4.114020,15.776638
1,6.314154,2.909432,5.428003,2.281814,17.249650
2,2.402763,9.534575,6.458104,0.178572,14.919508
3,9.961258,7.351945,7.219425,4.104965,20.229030
4,2.037802,9.621025,2.244710,0.477828,14.272082
...,...,...,...,...,...
995,2.658250,5.245613,9.591574,2.058272,17.469982
996,6.672634,8.605599,8.763929,6.766465,20.828604
997,3.847030,9.344537,0.500198,9.514672,17.058109
998,3.092017,9.321674,4.503694,0.325089,15.360015


#### 2.3 Process C
This process has its own formula as ml models. To use this formula in gurobi it is necesary define the formula directly as constraint. So, the formula used here is the original formula that will be used as "model"

In [15]:
list_features_process_c = [
    "X3",
    "06",
]

list_target_process_c = ["Y2"]

In [16]:
##### FEATURES

# generate data features process_c
np.random.seed(62)
values_features_process_c = 10 * np.random.rand(len_data, len(list_features_process_c))

In [17]:
#### TARGET

# define formula to map features into a target. np.exp(-1/data) + np.log(data)
formula_target_process_c = 24*1.125*values_features_process_c[:, 0] + values_features_process_c[:, 1]

# generate noise to add to the formula of the target generated
np.random.seed(12)
noise_target_process_c = 10 * np.random.rand(len_data, len(list_target_process_c))

# generate data target process_c
values_target_process_c  = formula_target_process_c.reshape(formula_target_process_c.shape[0], 1) + noise_target_process_c

In [18]:
# generate dataframe process_c
data_process_c = pd.DataFrame(values_features_process_c, columns = list_features_process_c)
data_process_c[list_target_process_c] = values_target_process_c
data_process_c

,X3,06,Y2
0,0.337547,4.891075,15.546475
1,8.460851,4.114020,239.957493
2,6.314154,2.909432,176.024733
3,5.428003,2.281814,154.175286
4,2.402763,9.534575,74.554934
...,...,...,...
995,7.257673,4.198883,206.805637
996,0.381145,2.181244,12.576272
997,4.243785,0.422370,123.687251
998,5.810263,1.745521,166.249236


### 3. Evaluate capacity of ml model (optional)
Evaluate the capacity of a Machine Learning model to predict this data. For this example it will use only a simple linear regression

#### 3.1 Process A

In [19]:
# list features-target
features_train_process_a = data_process_a.columns.tolist()[0:-1]
target_process_a = data_process_a.columns.tolist()[-1]

# generate data
X_process_a = data_process_a[features_train_process_a]
y_process_a = data_process_a[target_process_a]

# split train test
X_train_process_a, X_test_process_a, y_train_process_a, y_test_process_a = train_test_split(X_process_a,
                                                                                            y_process_a,
                                                                                            test_size = 0.2,
                                                                                            random_state = 42
                                                                                           )

# train model
lr_process_a = LinearRegression()
lr_process_a.fit(X_train_process_a, y_train_process_a)

# evaluate
lr_process_a.score(X_test_process_a, y_test_process_a)

0.3473543857086998

#### 3.2 Process B

In [20]:
# list features-target
features_train_process_b = data_process_b.columns.tolist()[0:-1]
target_process_b = data_process_b.columns.tolist()[-1]

# generate data
X_process_b = data_process_b[features_train_process_b]
y_process_b = data_process_b[target_process_b]

# split train test
X_train_process_b, X_test_process_b, y_train_process_b, y_test_process_b = train_test_split(X_process_b,
                                                                                            y_process_b,
                                                                                            test_size = 0.2,
                                                                                            random_state = 42
                                                                                           )

# train model
lr_process_b = LinearRegression()
lr_process_b.fit(X_train_process_b, y_train_process_b)

# evaluate
lr_process_b.score(X_test_process_b, y_test_process_b)

0.7963128209137649

#### 3.3 Process C

In [21]:
# list features-target
features_train_process_c = data_process_c.columns.tolist()[0:-1]
target_process_c = data_process_c.columns.tolist()[-1]

# generate data
X_process_c = data_process_c[features_train_process_c]
y_process_c = data_process_c[target_process_c]

# split train test
X_train_process_c, X_test_process_c, y_train_process_c, y_test_process_c = train_test_split(X_process_c,
                                                                                            y_process_c,
                                                                                            test_size = 0.2,
                                                                                            random_state = 42
                                                                                           )

# train model
lr_process_c = LinearRegression()
lr_process_c.fit(X_train_process_c, y_train_process_c)

# evaluate
lr_process_c.score(X_test_process_c, y_test_process_c)

0.9983920156337379

### 4. Save Data

#### 4.1 Process A

In [22]:
### save data TRAIN - TEST
name_model = 'process_a'


# ---
# save X_train
path_X_train = f'artifacts/data_training/{name_model}/X_train.pkl'
with open(path_X_train, "wb") as output:
    pickle.dump(X_train_process_a, output)
    output.close()

# save y_train
path_y_train = f'artifacts/data_training/{name_model}/y_train.pkl'
with open(path_y_train, "wb") as output:
    pickle.dump(y_train_process_a, output)
    output.close()


# ---
# save X_test
path_X_test = f'artifacts/data_training/{name_model}/X_test.pkl'
with open(path_X_test, "wb") as output:
    pickle.dump(X_test_process_a, output)
    output.close()

# save y_test
path_y_test = f'artifacts/data_training/{name_model}/y_test.pkl'
with open(path_y_test, "wb") as output:
    pickle.dump(y_test_process_a, output)
    output.close()

#### 4.2 Process B

In [23]:
### save data TRAIN - TEST
name_model = 'process_b'


# ---
# save X_train
path_X_train = f'artifacts/data_training/{name_model}/X_train.pkl'
with open(path_X_train, "wb") as output:
    pickle.dump(X_train_process_b, output)
    output.close()

# save y_train
path_y_train = f'artifacts/data_training/{name_model}/y_train.pkl'
with open(path_y_train, "wb") as output:
    pickle.dump(y_train_process_b, output)
    output.close()


# ---
# save X_test
path_X_test = f'artifacts/data_training/{name_model}/X_test.pkl'
with open(path_X_test, "wb") as output:
    pickle.dump(X_test_process_b, output)
    output.close()

# save y_test
path_y_test = f'artifacts/data_training/{name_model}/y_test.pkl'
with open(path_y_test, "wb") as output:
    pickle.dump(y_test_process_b, output)
    output.close()

#### 4.3 Process C

In [24]:
### save data TRAIN - TEST
name_model = 'process_c'


# ---
# save X_train
path_X_train = f'artifacts/data_training/{name_model}/X_train.pkl'
with open(path_X_train, "wb") as output:
    pickle.dump(X_train_process_c, output)
    output.close()

# save y_train
path_y_train = f'artifacts/data_training/{name_model}/y_train.pkl'
with open(path_y_train, "wb") as output:
    pickle.dump(y_train_process_c, output)
    output.close()


# ---
# save X_test
path_X_test = f'artifacts/data_training/{name_model}/X_test.pkl'
with open(path_X_test, "wb") as output:
    pickle.dump(X_test_process_c, output)
    output.close()

# save y_test
path_y_test = f'artifacts/data_training/{name_model}/y_test.pkl'
with open(path_y_test, "wb") as output:
    pickle.dump(y_test_process_c, output)
    output.close()